In [2]:
!CUDA_VISIBLE_DEVICES=0 python3 ./ocr_kor/deep-text-recognition-benchmark/test.py \
--eval_data ./data/test/img/ --benchmark_all_eval \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--saved_model ./saved_models/best_accuracy.pth

python3: can't open file './deep-text-recognition-benchmark/test.py': [Errno 2] No such file or directory


In [8]:
output = !CUDA_VISIBLE_DEVICES=0 python3 ./ocr_kor/deep-text-recognition-benchmark/demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder ./data/test/img/ \
--saved_model ./saved_models/best_accuracy.pth

In [9]:
output

['model input parameters 32 400 20 1 512 256 974 50 TPS ResNet BiLSTM Attn',
 'loading pretrained model from ./saved_models/best_accuracy.pth',
 'Traceback (most recent call last):',
 '  File "./ocr_kor/deep-text-recognition-benchmark/demo.py", line 117, in <module>',
 '    demo(opt)',
 '  File "./ocr_kor/deep-text-recognition-benchmark/demo.py", line 34, in demo',
 '    model.load_state_dict(torch.load(opt.saved_model))',
 '  File "/opt/conda/lib/python3.6/site-packages/torch/serialization.py", line 368, in load',
 '    return _load(f, map_location, pickle_module)',
 '  File "/opt/conda/lib/python3.6/site-packages/torch/serialization.py", line 542, in _load',
 '    result = unpickler.load()',
 '  File "/opt/conda/lib/python3.6/site-packages/torch/serialization.py", line 505, in persistent_load',
 '    data_type(size), location)',
 '  File "/opt/conda/lib/python3.6/site-packages/torch/serialization.py", line 114, in default_restore_location',
 '    result = fn(storage, location)',
 '  

In [5]:
!pip install pytorch_transformers

     |████████████████████████████████| 176 kB 578 kB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 3.1 MB/s eta 0:00:01
     |████████████████████████████████| 129 kB 5.2 MB/s eta 0:00:01
     |████████████████████████████████| 883 kB 5.5 MB/s eta 0:00:01
     |████████████████████████████████| 69 kB 2.6 MB/s eta 0:00:01
     |████████████████████████████████| 6.5 MB 3.1 MB/s eta 0:00:01     |███▏                            | 634 kB 3.1 MB/s eta 0:00:02     |█████████████                   | 2.6 MB 3.1 MB/s eta 0:00:02
     |████████████████████████████████| 547 kB 4.0 MB/s eta 0:00:01
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893258 sha256=d55e0e7cb1e6c6d78cd31c60852db1e3071607a2ea50a823ecda48d5e6aefb33
  Stored in directory: /root/.cache/pip/wheels/49/25/98/cdea9c79b2d9a22ccc59540b1784b67f06b633378e97f58da2
Successfully built sacremoses
